In [ ]:
# Cell 1: Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import os

print("Torch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
# Cell 2: Setup Paths (Relative - works for everyone)
train_dir = "../data/Combined Dataset/train"
test_dir = "../data/Combined Dataset/test"
model_path = "../app/alz_resnet18.pt"

print(f"Train dir exists: {os.path.exists(train_dir)}")
print(f"Test dir exists: {os.path.exists(test_dir)}")
print(f"Model file exists: {os.path.exists(model_path)}")

In [ ]:
# Cell 3: Data Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Transforms ready")

In [ ]:
# Cell 4: Load Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

print(f"Training images: {len(train_dataset)}")
print(f"Test images: {len(test_dataset)}")
print(f"Classes: {train_dataset.class_to_idx}")

In [ ]:
# Cell 5: Define Model Architecture
def create_model():
    """Create ResNet18 model with 4 output classes"""
    model = models.resnet18(pretrained=True)
    
    # Freeze all layers initially
    for param in model.parameters():
        param.requires_grad = False
    
    # Unfreeze last two blocks for fine-tuning
    for param in model.layer3.parameters():
        param.requires_grad = True
    
    for param in model.layer4.parameters():
        param.requires_grad = True
    
    # Replace final layer for 4 classes
    model.fc = nn.Linear(model.fc.in_features, 4)
    
    return model.to(device)

model = create_model()
print("Model architecture ready")

---
## 🔒 Load Existing Model or Train New One
**By default, this loads the existing trained model. Only set TRAIN=True if you need to retrain.**

In [ ]:
# Cell 6: Load or Train Model
TRAIN = False  # ⚠️ Set to True ONLY if you want to retrain from scratch

if os.path.exists(model_path) and not TRAIN:
    print("✅ Loading existing trained model...")
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print(f"Model loaded from {model_path}")
else:
    if TRAIN:
        print("⚠️ WARNING: Training new model (this will overwrite existing model!)")
    else:
        print("Model file not found. Training new model...")
    
    # Setup training
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=1e-5
    )
    
    # Training loop
    num_epochs = 5
    print(f"Training for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")
    
    # Save trained model
    torch.save(model.state_dict(), model_path)
    print(f"\n✅ Model saved to {model_path}")

---
## 📊 Model Evaluation
**This section evaluates the model on the test set and prints metrics.**

In [ ]:
# Cell 7: Prepare Test DataLoader
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
print(f"Test loader ready with {len(test_loader)} batches")

In [ ]:
# Cell 8: Run Evaluation
model.eval()
all_preds = []
all_labels = []

print("Running evaluation on test set...")

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(f"Evaluated {len(all_labels)} test images")

In [ ]:
# Cell 9: Calculate Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
cm = confusion_matrix(all_labels, all_preds)

# Class names
class_names = ['Mild Impairment', 'Moderate Impairment', 'No Impairment', 'Very Mild Impairment']

print("="*50)
print("📊 MODEL EVALUATION RESULTS")
print("="*50)
print(f"\n✅ Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"✅ F1-Score (weighted): {f1:.4f}")
print("\n" + "="*50)
print("📋 Confusion Matrix:")
print("="*50)
print(cm)
print("\nRows = True Labels, Columns = Predicted Labels")
print(f"Classes: {class_names}")

In [ ]:
# Cell 10: Detailed Classification Report
print("\n" + "="*50)
print("📈 DETAILED CLASSIFICATION REPORT")
print("="*50)
print(classification_report(all_labels, all_preds, target_names=class_names))

In [ ]:
# Cell 11: Per-Class Accuracy
print("\n" + "="*50)
print("🎯 PER-CLASS ACCURACY")
print("="*50)

for i, class_name in enumerate(class_names):
    class_correct = cm[i, i]
    class_total = cm[i].sum()
    class_acc = class_correct / class_total if class_total > 0 else 0
    print(f"{class_name:25s}: {class_acc:.4f} ({class_acc*100:.2f}%) - {class_correct}/{class_total} correct")

---
## ✅ Summary

This notebook:
- ✅ Uses **relative paths** (reproducible across machines)
- ✅ Loads **existing trained model** by default (safe)
- ✅ Includes **training guard** to prevent accidental retraining
- ✅ Evaluates on **test set**
- ✅ Prints **accuracy, F1-score, confusion matrix**
- ✅ Saves model to **`../app/alz_resnet18.pt`**

**Ready for your report! 🎉**